In [1]:
import sys
sys.path.append("/data6/shubham/PC/scripts/BrainDiVE/modfit/code")

import os
import numpy as np
import h5py
from scipy.io import loadmat
import PIL.Image
import nibabel as nib
import pandas as pd
import pickle
import time
from utils import default_paths, roi_utils, prf_utils
from model_fitting import initialize_fitting
import gc
import pickle as pkl
import pandas as pd

import imageio
import os
from skimage.transform import rescale, resize
import PIL
from PIL import Image
import h5py

import cortex
# from cortex import config  # Not used anywhere
import numpy as np
# from plotter_utils import volume_maker, visualize_volume, view_1, view_2, make_image, close_handle
import matplotlib.pyplot as plt
import os
os.getcwd()

import nibabel as nib


def load_from_nii(nii_file):
    return nib.load(nii_file).get_fdata()


nsd_root = default_paths.nsd_root;
stim_root = default_paths.stim_root
beta_root = default_paths.beta_root

In [8]:
def get_subj_df(subject):
    """
    Get info about the 10,000 images that were shown to each subject.
    Note this is not the full ordered sequence of trials (which is 30,000 long)
    This is only the unique images 
    (matches what is in /user_data/mmhender/nsd_stimuli/stimuli/nsd/S1_stimuli....h5py)
    """
    exp_design_file = os.path.join(nsd_root,"nsddata/experiments/nsd/nsd_expdesign.mat")
    exp_design = loadmat(exp_design_file)
    subject_idx  = exp_design['subjectim']
    
    nsd_meta_file = os.path.join(nsd_root, 'nsddata/experiments/nsd/nsd_stim_info_merged.pkl')
    with open(nsd_meta_file,'rb') as f:
        stim_info = pickle.load(f,encoding="latin1")
    
    ss=subject-1
    subject_df = stim_info.loc[subject_idx[ss,:]-1]

    return subject_df

def get_master_image_order():    
    """
    Gather the "ordering" information for NSD images.
    masterordering gives zero-indexed ordering of indices (matlab-like to python-like), same for all subjects. 
    consists of 30000 values in the range [0-9999], which provide a list of trials in order. 
    The value in ordering[ii] tells the index into the subject-specific stimulus array that we would need to take to
    get the image for that trial.
    """
    # exp_design_file = "/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_expdesign.mat"

    # exp_design_file = os.path.join(nsd_root, 'nsddata','experiments','nsd','nsd_expdesign.mat')
    exp_design_file = "/data6/shubham/PC/data/BrinDiVE_fromPrince/nsd_data/nsddata/experiments/nsd/nsd_expdesign.mat"
    exp_design = loadmat(exp_design_file)
    
    image_order = exp_design['masterordering'].flatten() - 1 
    
    return image_order

def load_image_data_partitions(subject):
    
    fn2load = "/data6/shubham/PC/data/results_preprocessed/Image_data_partitions.npy"
    print()
    print('loading train/holdout/val image list from %s'%fn2load)
    partitions = np.load(fn2load, allow_pickle=True).item()
    is_trn = partitions['is_trn'][:,subject-1]
    is_holdout = partitions['is_holdout'][:,subject-1]
    is_val = partitions['is_val'][:,subject-1]
    
    return is_trn, is_holdout, is_val


In [3]:
subject = 1
subj_df = get_subj_df(subject)

In [4]:
subj_df

,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.093750,2950,False,True,True,1,1,...,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566
2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,2990,False,True,True,1,1,...,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711
3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.000000,3049,False,True,True,1,1,...,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697
3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,3077,False,True,True,1,1,...,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537
3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,3146,False,True,True,1,1,...,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,464089,val2017,"(0, 0, 0.125, 0.125)",0.142857,1913,False,False,False,1,0,...,0,0,0,0,0,0,0,0,0,0
15168,36736,train2017,"(0, 0, 0.07, 0.07)",0.133333,15168,False,False,False,1,0,...,0,0,0,0,0,0,0,0,0,0
19471,49942,train2017,"(0, 0, 0.12421875, 0.12421875)",0.000000,19471,False,False,False,1,0,...,0,0,0,0,0,0,0,0,0,0
49507,426815,train2017,"(0, 0, 0.1609375, 0.1609375)",0.000000,49507,False,False,False,1,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
image_order = get_master_image_order()
image_order = np.unique(image_order)


In [13]:
len(image_order)

10000

In [23]:
is_shared_image = np.array(subj_df['shared1000'])
shared_1000_inds = is_shared_image[image_order]
val_inds = shared_1000_inds

In [33]:
shared_1000_inds

array([ True,  True,  True, ..., False, False, False])

In [35]:
is_trn, is_holdout, is_val = load_image_data_partitions(subject)


loading train/holdout/val image list from /data6/shubham/PC/data/results_preprocessed/Image_data_partitions.npy


In [37]:
len(is_trn)

10000

In [34]:

is_trn = is_trn[image_order]
is_val = is_val[image_order]
is_holdout = is_holdout[image_order]
assert(np.all(is_val==val_inds))
holdout_inds = is_holdout


loading train/holdout/val image list from /data6/shubham/PC/data/results_preprocessed/Image_data_partitions.npy


In [18]:
a = np.array([1, 0, 0, 1, 0, 0, 1, 0, 1], dtype = bool)
b = np.arange(10, 19)
c = a[b]
c

IndexError: index 10 is out of bounds for axis 0 with size 9